In [10]:
import torch
import torch.utils.data as data
import pandas as pd

In [114]:
from data import DigitsDateset
from models.DigitsRecgnitionNetwork import DigitsRecognizer, DigitsPredictor
from utils import multi_target_loss, sum_list_strs, show

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
ds = DigitsDateset('train')
net = DigitsRecognizer()
net.to(device)
dl = data.DataLoader(dataset=ds, batch_size=16, shuffle=True)

In [28]:
net.train()
optim = torch.optim.Adam(net.parameters(), lr=0.0001)
for _ in range(5):
    mean_loss = 0
    for i, (X, y) in enumerate(dl):
        optim.zero_grad()
        X, y = X.to(device), y.to(device)
        loss = multi_target_loss(y, net(X))
        loss.backward()
        optim.step()
        mean_loss += loss.item()
        if (i + 1) % 10 == 0:
            print(mean_loss / 10)
            mean_loss = 0

0.038842447381466624
0.025389123940840364
0.019263003673404454
0.013371636206284166
0.012309118243865668
0.01942392559722066
0.010057745198719204
0.009032084909267723
0.006948444829322398
0.006838354980573058
0.007887748000212014
0.009189618495292961
0.006269496609456837
0.004346975835505873
0.004852717882022262
0.0065213748253881935
0.003042995277792215
0.006493911007419228
0.0034969549567904325
0.005363784061046317
0.003921570209786296
0.005045577441342175
0.004564128420315683
0.00475455328123644
0.0039665480027906595
0.004499282193137333
0.003064467734657228
0.004474163393024355
0.004998580378014594
0.0027022039284929633


In [30]:
# test accuracy
# TODO implement it better
with torch.no_grad():
    net.eval()
    net.to(device)
    total_acc = 0
    for X, y in dl:
        X, y = X.to(device), y.to(device)
        logits = net(X)
        total_acc += (logits.argmax(dim=2) == y).all(dim=1).sum()
    print(total_acc / 1000)

tensor(1., device='cuda:0')


In [115]:
with torch.no_grad():
    net.eval()
    net.to(device)
    test_ds = DigitsDateset('test')
    test_dl = data.DataLoader(test_ds, batch_size=16, shuffle=False)
    predictions = []
    for X in test_dl:
        X = X.to(device)
        logits = net(X)
        predictions += DigitsPredictor()(logits)

    # make submission file
    submission = pd.DataFrame(columns=['filename', 'result'])
    submission['filename'] = [f'test_{i + 1}.jpg' for i in range(500)]
    # TODO: add comments here
    submission['result'] = [sum_list_strs(list_strs)[0: 5] for list_strs in predictions]
    submission.to_csv(f'./submission.csv', header=True, index=False)

In [125]:
submission['result'].sample(10)

441    00291
237    00233
352    00834
67     00334
257    00652
400    00267
344    00811
42     00410
343    00426
495    00675
166    00416
258    00507
62     02189
197    00611
359    00474
196    00167
357    00567
270    00167
53     00501
184    00849
Name: result, dtype: object